# Helper notebook to help parse and analyze results from the experiments

In [ ]:
from tqdm import tqdm 
from sklearn.metrics import roc_curve
import numpy as np
from scipy.stats import mode
import random

def compute_eer(label, pred):
    """
    Computes EER given a list of labels and predictions.

    Code inspired by https://github.com/YuanGongND/python-compute-eer
    """
    # all fpr, tpr, fnr, fnr, threshold are lists (in the format of np.array)
    fpr, tpr, threshold = roc_curve(label, pred)
    fnr = 1 - tpr

    # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]

    # return the mean of eer from fpr and from fnr
    eer = (eer_1 + eer_2) / 2
    return eer


def calculate_eer_in_sliding_window(y_test, res, window_size=1):
    '''
    Computes the EER using a window of size `window_size`
    '''
    frrs = []
    fars = []
    end_index = max(np.where(y_test == 1)[0])
    user_results = res[: end_index]
    intruder_results = res[end_index:]

    user_windows = user_results[(
        np.expand_dims(np.arange(window_size), 0) +
        np.expand_dims(np.arange(len(user_results) - window_size), 0).T
    )]

    intruder_windows = intruder_results[(
            np.expand_dims(np.arange(window_size), 0) +
            np.expand_dims(np.arange(len(intruder_results) - window_size), 0).T
        )]

    desc_scores = np.sort(res)[::-1]
    thresholds = []
    for threshold in desc_scores[::10]:
        FRR = 1 - np.mean(mode(user_windows >= threshold, axis=1)[0])
        FAR = 1 - np.mean(mode(intruder_windows < threshold, axis=1)[0])

        thresholds.append(threshold)
        frrs.append(FRR)
        fars.append(FAR)

    fars = np.array(fars)
    frrs = np.array(frrs)
    thresholds = np.array(thresholds)

    eer = fars[np.argwhere(np.diff(np.sign(fars - frrs))).flatten()]
    thresh = thresholds[np.argwhere(np.diff(np.sign(fars - frrs))).flatten()]
    return (eer[0], thresh) if eer.size > 0 else (1, 0)

def calculate_eer_with_intruders(y_test, res, window_size=1, intruders = 1, threshold = 0):
    '''
    Calculates the EER by polluting teh data with `intruders` indtruders. threshold for the EER has to be provided. 
    '''
    end_index = max(np.where(y_test == 1)[0])
    user_results = res[: end_index]
    intruder_results = res[end_index:]

    user_windows = user_results[(
        np.expand_dims(np.arange(window_size), 0) +
        np.expand_dims(np.arange(len(user_results) - window_size), 0).T
    )]

    intruder_windows = intruder_results[(
            np.expand_dims(np.arange(intruders), 0) +
            np.expand_dims(np.arange(len(intruder_results) - window_size), 0).T
        )]

    start_index = random.randint(0, intruder_windows.shape[0] - user_windows.shape[0] - 2)
    intruder_windows = intruder_windows[start_index: start_index + user_windows.shape[0]]
    if intruders != 0:
        user_windows[:, -intruders:] = intruder_windows

    return np.mean(mode(user_windows > threshold, axis=1)[0])

# Calculate results from a specific experiment (for all iterations)

In [8]:
import pickle, glob

results = []
for file in tqdm(glob.glob(f'test_results/*/*')):
    with open(file, 'rb') as f:
        y_test, res, *_ = pickle.load(f)
    results.append(compute_eer(y_test, res))
print(np.mean(results, axis=0))

100%|██████████| 41/41 [00:00<00:00, 625.80it/s]

[0.30608795]
